In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
data=pd.read_csv('/content/modified_data.csv') #grab the dataset

In [ ]:
data.head() #view the dataset

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,0,12,2008,WD,Normal,250000


#Step Two:  Prepare Dataset
#Search For Missing Value


In [ ]:
data.drop('Id', axis=1,inplace=True)

In [ ]:
data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,0,12,2008,WD,Normal,250000


In [ ]:
data.shape #check the dataset 1460-rows 76 -columns


(1460, 76)

In [ ]:
data.isna().any()

MSSubClass       False
MSZoning         False
LotFrontage       True
LotArea          False
Street           False
                 ...  
MoSold           False
YrSold           False
SaleType         False
SaleCondition    False
SalePrice        False
Length: 76, dtype: bool

In [ ]:
original_features=list(data.columns)
original_features

['MSSubClass',
 'MSZoning',
 'LotFrontage',
 'LotArea',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch

In [ ]:
features_with_missing=list(data.columns[data.isna().any()])
features_with_missing
len(features_with_missing)

15

In [ ]:
original_features=list(data.columns)
original_features
len(original_features)

76

In [ ]:
for feature in features_with_missing:
  print(feature,' ',(data[feature].isna().sum())/len(data)*100)

LotFrontage   17.73972602739726
MasVnrType   0.547945205479452
MasVnrArea   0.547945205479452
BsmtQual   2.5342465753424657
BsmtCond   2.5342465753424657
BsmtExposure   2.6027397260273974
BsmtFinType1   2.5342465753424657
BsmtFinType2   2.6027397260273974
Electrical   0.0684931506849315
FireplaceQu   47.26027397260274
GarageType   5.5479452054794525
GarageYrBlt   5.5479452054794525
GarageFinish   5.5479452054794525
GarageQual   5.5479452054794525
GarageCond   5.5479452054794525


In [ ]:
data.dropna(axis=1,thresh=0.5*(len(data)),inplace=True) #drop feature with missing values

In [ ]:
mean_fill=['LotFrontage', 'MasVnrArea'] #fill using mean
bfill=['FireplaceQu'] #backward fill
ffill=list(set(features_with_missing)-set(mean_fill)-set(bfill)) #forward fill

In [ ]:
ffill

['GarageYrBlt',
 'BsmtQual',
 'BsmtCond',
 'MasVnrType',
 'BsmtFinType2',
 'BsmtExposure',
 'Electrical',
 'GarageQual',
 'GarageFinish',
 'GarageCond',
 'BsmtFinType1',
 'GarageType']

In [ ]:
for feature in mean_fill:
  data[feature].fillna(data[feature].mean(),inplace=True)

In [ ]:
for feature in ffill:
  data[feature].fillna(method='ffill',inplace=True)

In [ ]:
for feature in bfill:
  data[feature].fillna(method='bfill',inplace=True)

In [ ]:
data.isna().any()

MSSubClass       False
MSZoning         False
LotFrontage      False
LotArea          False
Street           False
                 ...  
MoSold           False
YrSold           False
SaleType         False
SaleCondition    False
SalePrice        False
Length: 76, dtype: bool


#Encode the Dataset



In [ ]:
cat_data=data.select_dtypes(include=['object'])
cat_data.head(2) #viewing the categorical features which are features without numbers

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
1,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,Gable,CompShg,MetalSd,MetalSd,None,TA,TA,CBlock,Gd,TA,Gd,ALQ,Unf,GasA,Ex,Y,SBrkr,TA,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal


In [ ]:
cat_features=list(cat_data.columns)
cat_features #features that need encoding

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [ ]:
nominal=['MSZoning', 'LandContour','LotConfig','Neighborhood']
ordinal=list(set(cat_features)-set(nominal))
numerical=list(set(original_features)-set(cat_features))
target=['SalePrice']

In [ ]:
data[nominal]

,MSZoning,LandContour,LotConfig,Neighborhood
0,RL,Lvl,Inside,CollgCr
1,RL,Lvl,FR2,Veenker
2,RL,Lvl,Inside,CollgCr
3,RL,Lvl,Corner,Crawfor
4,RL,Lvl,FR2,NoRidge
...,...,...,...,...
1455,RL,Lvl,Inside,Gilbert
1456,RL,Lvl,Inside,NWAmes
1457,RL,Lvl,Inside,Crawfor
1458,RL,Lvl,Inside,NAmes


In [ ]:
data[numerical]

,BedroomAbvGr,2ndFlrSF,TotalBsmtSF,BsmtFinSF2,BsmtFullBath,TotRmsAbvGrd,LowQualFinSF,WoodDeckSF,FullBath,1stFlrSF,MasVnrArea,GarageArea,LotFrontage,MiscVal,PoolArea,MSSubClass,YearRemodAdd,GrLivArea,3SsnPorch,OpenPorchSF,BsmtUnfSF,YearBuilt,OverallQual,KitchenAbvGr,SalePrice,EnclosedPorch,BsmtFinSF1,GarageYrBlt,Fireplaces,MoSold,OverallCond,BsmtHalfBath,LotArea,HalfBath,ScreenPorch,GarageCars,YrSold
0,3,854,856,0,1,8,0,0,2,856,196.0,548,65.0,0,0,60,2003,1710,0,61,150,2003,7,1,208500,0,706,2003.0,0,2,5,0,8450,1,0,2,2008
1,3,0,1262,0,0,6,0,298,2,1262,0.0,460,80.0,0,0,20,1976,1262,0,0,284,1976,6,1,181500,0,978,1976.0,1,5,8,1,9600,0,0,2,2007
2,3,866,920,0,1,6,0,0,2,920,162.0,608,68.0,0,0,60,2002,1786,0,42,434,2001,7,1,223500,0,486,2001.0,1,9,5,0,11250,1,0,2,2008
3,3,756,756,0,1,7,0,0,1,961,0.0,642,60.0,0,0,70,1970,1717,0,35,540,1915,7,1,140000,272,216,1998.0,1,2,5,0,9550,0,0,3,2006
4,4,1053,1145,0,1,9,0,192,2,1145,350.0,836,84.0,0,0,60,2000,2198,0,84,490,2000,8,1,250000,0,655,2000.0,1,12,5,0,14260,1,0,3,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,3,694,953,0,0,7,0,0,2,953,0.0,460,62.0,0,0,60,2000,1647,0,40,953,1999,6,1,175000,0,0,1999.0,1,8,5,0,7917,1,0,2,2007
1456,3,0,1542,163,1,7,0,349,2,2073,119.0,500,85.0,0,0,20,1988,2073,0,0,589,1978,6,1,210000,0,790,1978.0,2,2,6,0,13175,0,0,2,2010
1457,4,1152,1152,0,0,9,0,0,2,1188,0.0,252,66.0,2500,0,70,2006,2340,0,60,877,1941,7,1,266500,0,275,1941.0,2,5,9,0,9042,0,0,1,2010
1458,2,0,1078,1029,1,5,0,366,1,1078,0.0,240,68.0,0,0,20,1996,1078,0,0,0,1950,5,1,142125,112,49,1950.0,0,4,6,0,9717,0,0,1,2010


In [ ]:
for feature in ordinal: 
  data[feature]=(data[feature].astype('category')).cat.codes

In [ ]:
df_nominal=pd.get_dummies(data[nominal]) #one hot encoding

In [ ]:
df_ordinal=data[ordinal]


In [ ]:
df_numerical=data[numerical]

In [ ]:
encoded_data=pd.concat([df_numerical, df_nominal, df_ordinal], axis=1)

In [ ]:
encoded_data.head(3)

,BedroomAbvGr,2ndFlrSF,TotalBsmtSF,BsmtFinSF2,BsmtFullBath,TotRmsAbvGrd,LowQualFinSF,WoodDeckSF,FullBath,1stFlrSF,MasVnrArea,GarageArea,LotFrontage,MiscVal,PoolArea,MSSubClass,YearRemodAdd,GrLivArea,3SsnPorch,OpenPorchSF,BsmtUnfSF,YearBuilt,OverallQual,KitchenAbvGr,SalePrice,EnclosedPorch,BsmtFinSF1,GarageYrBlt,Fireplaces,MoSold,OverallCond,BsmtHalfBath,LotArea,HalfBath,ScreenPorch,GarageCars,YrSold,MSZoning,LandContour,LotConfig,Neighborhood,HeatingQC,SaleCondition,ExterCond,Utilities,BsmtFinType2,ExterQual,Condition2,BsmtFinType1,GarageType,BldgType,RoofMatl,Foundation,LotShape,BsmtQual,KitchenQual,BsmtCond,LandSlope,MasVnrType,Functional,Heating,CentralAir,GarageQual,GarageFinish,Condition1,SaleType,HouseStyle,GarageCond,RoofStyle,Exterior1st,Exterior2nd,Street,BsmtExposure,PavedDrive,Electrical,FireplaceQu
0,3,854,856,0,1,8,0,0,2,856,196.0,548,65.0,0,0,60,2003,1710,0,61,150,2003,7,1,208500,0,706,2003.0,0,2,5,0,8450,1,0,2,2008,3,3,4,5,0,4,4,0,5,2,2,2,1,0,1,2,3,2,2,3,0,1,6,1,1,4,1,2,8,5,4,1,12,13,1,3,2,4,5
1,3,0,1262,0,0,6,0,298,2,1262,0.0,460,80.0,0,0,20,1976,1262,0,0,284,1976,6,1,181500,0,978,1976.0,1,5,8,1,9600,0,0,2,2007,3,3,2,24,0,4,4,0,5,3,2,0,1,0,1,1,3,2,3,3,0,2,6,1,1,4,1,1,8,2,4,1,8,8,1,1,2,4,5
2,3,866,920,0,1,6,0,0,2,920,162.0,608,68.0,0,0,60,2002,1786,0,42,434,2001,7,1,223500,0,486,2001.0,1,9,5,0,11250,1,0,2,2008,3,3,4,5,0,4,4,0,5,2,2,2,1,0,1,2,0,2,2,3,0,1,6,1,1,4,1,2,8,5,4,1,12,13,1,2,2,4,5


#Data Normalization

In [ ]:
X=encoded_data.drop('SalePrice', axis=1)

In [ ]:
y=data[target]
y

,SalePrice
0,208500
1,181500
2,223500
3,140000
4,250000
...,...
1455,175000
1456,210000
1457,266500
1458,142125


In [ ]:
X.to_numpy()
X

,BedroomAbvGr,2ndFlrSF,TotalBsmtSF,BsmtFinSF2,BsmtFullBath,TotRmsAbvGrd,LowQualFinSF,WoodDeckSF,FullBath,1stFlrSF,MasVnrArea,GarageArea,LotFrontage,MiscVal,PoolArea,MSSubClass,YearRemodAdd,GrLivArea,3SsnPorch,OpenPorchSF,BsmtUnfSF,YearBuilt,OverallQual,KitchenAbvGr,EnclosedPorch,BsmtFinSF1,GarageYrBlt,Fireplaces,MoSold,OverallCond,BsmtHalfBath,LotArea,HalfBath,ScreenPorch,GarageCars,YrSold,MSZoning,LandContour,LotConfig,Neighborhood,HeatingQC,SaleCondition,ExterCond,Utilities,BsmtFinType2,ExterQual,Condition2,BsmtFinType1,GarageType,BldgType,RoofMatl,Foundation,LotShape,BsmtQual,KitchenQual,BsmtCond,LandSlope,MasVnrType,Functional,Heating,CentralAir,GarageQual,GarageFinish,Condition1,SaleType,HouseStyle,GarageCond,RoofStyle,Exterior1st,Exterior2nd,Street,BsmtExposure,PavedDrive,Electrical,FireplaceQu
0,3,854,856,0,1,8,0,0,2,856,196.0,548,65.0,0,0,60,2003,1710,0,61,150,2003,7,1,0,706,2003.0,0,2,5,0,8450,1,0,2,2008,3,3,4,5,0,4,4,0,5,2,2,2,1,0,1,2,3,2,2,3,0,1,6,1,1,4,1,2,8,5,4,1,12,13,1,3,2,4,5
1,3,0,1262,0,0,6,0,298,2,1262,0.0,460,80.0,0,0,20,1976,1262,0,0,284,1976,6,1,0,978,1976.0,1,5,8,1,9600,0,0,2,2007,3,3,2,24,0,4,4,0,5,3,2,0,1,0,1,1,3,2,3,3,0,2,6,1,1,4,1,1,8,2,4,1,8,8,1,1,2,4,5
2,3,866,920,0,1,6,0,0,2,920,162.0,608,68.0,0,0,60,2002,1786,0,42,434,2001,7,1,0,486,2001.0,1,9,5,0,11250,1,0,2,2008,3,3,4,5,0,4,4,0,5,2,2,2,1,0,1,2,0,2,2,3,0,1,6,1,1,4,1,2,8,5,4,1,12,13,1,2,2,4,5
3,3,756,756,0,1,7,0,0,1,961,0.0,642,60.0,0,0,70,1970,1717,0,35,540,1915,7,1,272,216,1998.0,1,2,5,0,9550,0,0,3,2006,3,3,0,6,2,0,4,0,5,3,2,0,5,0,1,0,0,3,2,1,0,2,6,1,1,4,2,2,8,5,4,1,13,15,1,3,2,4,3
4,4,1053,1145,0,1,9,0,192,2,1145,350.0,836,84.0,0,0,60,2000,2198,0,84,490,2000,8,1,0,655,2000.0,1,12,5,0,14260,1,0,3,2008,3,3,2,15,0,4,4,0,5,2,2,2,1,0,1,2,0,2,2,3,0,1,6,1,1,4,1,2,8,5,4,1,12,13,1,0,2,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,3,694,953,0,0,7,0,0,2,953,0.0,460,62.0,0,0,60,2000,1647,0,40,953,1999,6,1,0,0,1999.0,1,8,5,0,7917,1,0,2,2007,3,3,4,8,0,4,4,0,5,3,2,5,1,0,1,2,3,2,3,3,0,2,6,1,1,4,1,2,8,5,4,1,12,13,1,3,2,4,5
1456,3,0,1542,163,1,7,0,349,2,2073,119.0,500,85.0,0,0,20,1988,2073,0,0,589,1978,6,1,0,790,1978.0,2,2,6,0,13175,0,0,2,2010,3,3,4,14,4,4,4,0,4,3,2,0,1,0,1,1,3,2,3,3,0,3,2,1,1,4,2,2,8,2,4,1,9,10,1,3,2,4,5
1457,4,1152,1152,0,0,9,0,0,2,1188,0.0,252,66.0,2500,0,70,2006,2340,0,60,877,1941,7,1,0,275,1941.0,2,5,9,0,9042,0,0,1,2010,3,3,4,6,0,4,2,0,5,0,2,2,1,0,1,4,3,3,2,1,0,2,6,1,1,4,1,2,8,5,4,1,5,5,1,3,2,4,3
1458,2,0,1078,1029,1,5,0,366,1,1078,0.0,240,68.0,0,0,20,1996,1078,0,0,0,1950,5,1,112,49,1950.0,0,4,6,0,9717,0,0,1,2010,3,3,4,12,2,4,4,0,4,3,2,2,1,0,1,1,3,3,2,3,0,2,6,1,1,4,2,2,8,2,4,3,8,8,1,2,2,0,0


In [ ]:
y=y.to_numpy() #change to a matrix
y

array([[208500],
       [181500],
       [223500],
       ...,
       [266500],
       [142125],
       [147500]])

In [ ]:
X.shape #rows and columns

(1460, 75)

In [ ]:
y.shape

(1460, 1)

In [ ]:
X=StandardScaler().fit_transform(X)
y=StandardScaler().fit_transform(y)

In [ ]:
y

array([[ 0.34727322],
       [ 0.00728832],
       [ 0.53615372],
       ...,
       [ 1.07761115],
       [-0.48852299],
       [-0.42084081]])

In [ ]:
X.shape

(1460, 75)

#PCA Dimensionality Reduction


In [ ]:
pca=PCA(n_components=2)
p_components=pca.fit_transform(X)

In [ ]:
p_components

array([[ 2.15078401,  0.05459886],
       [ 0.14094996, -1.72696945],
       [ 2.57197685, -0.07926865],
       ...,
       [ 1.33094974,  2.7218465 ],
       [-2.93359946, -2.47638569],
       [-0.78227828, -2.36910706]])

------------------------------------------------------

In [ ]:
pca.explained_variance_ratio_n 

array([0.13688131, 0.05387667])

You could do this or 👆🏾 this 👇🏾

In [ ]:
pca_var=PCA(0.9) #90%
new_pcs=pca_var.fit_transform(X)

In [ ]:
pca_var.n_components_

49

-------------------------------------------